In [1]:
from Module import *
from Optimizer import *
from DataTools import *

In [7]:
train_dataset = DataSet(['train-images-idx3-ubyte', 'train-labels-idx1-ubyte'], data_type='mnist')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


test_dataset = DataSet(['t10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte'], data_type='mnist')
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)

In [3]:
def ResidualMLP(input_dim, output_dim, num_blocks):
    net = Sequential(*[Residual(Linear(input_dim, input_dim), ReLU(), Linear(input_dim, input_dim)) for _ in range(num_blocks)],
                     Linear(input_dim, output_dim), Softmax())
    return net

In [4]:
net = ResidualMLP(784, 10, 3)
optimizer = SGD(net.parameters(), lr=0.01, batch_size=64)
scheduler = ConstantScheduler(optimizer=optimizer, lr=0.01)
loss_fn = CrossEntrophyLoss()

In [5]:
num_epochs = 2
for epoch in range(num_epochs):
    print(f"epcosh: {epoch}")
    for data, labels in train_loader:
        data = data.reshape(data.shape[0], -1)
        data = Tensor.make_const(data)
        optimizer.reset_grad()
        output = net(data)
        one_hot_vector = np.zeros((len(labels), 10))
        one_hot_vector[np.arange(len(labels)), labels] = 1
        labels = one_hot_vector
        labels = Tensor.make_const(labels)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

epcosh: 0


/home/robot/Workspace/nn-learngin/core.py:285: RuntimeWarning: overflow encountered in exp
  return np.exp(a)
/home/robot/Workspace/nn-learngin/core.py:291: RuntimeWarning: invalid value encountered in divide
  return a / b
/home/robot/Workspace/nn-learngin/core.py:313: RuntimeWarning: divide by zero encountered in log
  return np.log(a)
/home/robot/Workspace/nn-learngin/core.py:232: RuntimeWarning: invalid value encountered in multiply
  return a * b


epcosh: 1


In [30]:
for data, labels in test_loader:
    data = data.reshape(data.shape[0], -1)
    data = Tensor.make_const(data)
    output = net(data)
    print(output.data)
    print(f"pred : {np.argmax(output.data, axis=1)} \n")
    print(f"labels :{labels}")
    break

[[nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
pred : [0 0 0 0] 

labels :[6 3 5 4]
